Procesamiento de datos:
Se realiza la lectura en pandas de los archivos de entrenamiento y prueba

In [83]:
import pandas as pd

test_data = pd.read_csv(
    "../files/input/test_data.csv.zip",
    index_col=False,
    compression="zip",
)
train_data = pd.read_csv(
    "../files/input/train_data.csv.zip",
    index_col=False,
    compression="zip",
)

Renombramos las columnas 'default payment next month' tanto en los datos de entrenamiento como en los de prueba

In [84]:
# - Renombre la columna "default payment next month" a "default".
test_data = test_data.rename(columns={'default payment next month': 'default'})
train_data = train_data.rename(columns={'default payment next month': 'default'})

Eliminamos la columna 'ID' no aporta información útil 

In [85]:
# - Remueva la columna "ID".
test_data=test_data.drop(columns=['ID'])
train_data=train_data.drop(columns=['ID'])

Filtramos las filas donde MARRIAGE y EDUCATION no sea igual a '0' tanto en el conjunto de prueba como en el de entranamiento 

In [86]:
import numpy as np

# Para train_data
train_data = train_data.loc[train_data["MARRIAGE"] != 0]
train_data = train_data.loc[train_data["EDUCATION"] != 0]


# Para test_data (corregido)
test_data = test_data.loc[test_data["MARRIAGE"] != 0]
test_data = test_data.loc[test_data["EDUCATION"] != 0]

Para la columna EDUCATION, valores > 4 indican niveles superiores de educación, se agrupa estos valores en la categoría "others".

In [87]:

test_data['EDUCATION'] = test_data['EDUCATION'].apply(lambda x: 4 if x > 4 else x)
train_data['EDUCATION'] = train_data['EDUCATION'].apply(lambda x: 4 if x > 4 else x)

print(train_data["EDUCATION"].value_counts())
print(test_data["EDUCATION"].value_counts())


EDUCATION
2    9756
1    7476
3    3396
4     325
Name: count, dtype: int64
EDUCATION
2    4268
1    3105
3    1477
4     129
Name: count, dtype: int64


Separamos conjuntos de datos de entrenamiento y prueba 
tanto para los datos de entrenamiento y prueba, en las características o x_train y x_test se elimina la columna "default" ya que esta es el target o y_train y y_test

In [88]:
x_train=train_data.drop(columns="default")
y_train=train_data["default"]


x_test=test_data.drop(columns="default")
y_test=test_data["default"]

Realizamos un preprocesamiento de los datos, definimos variables categóricas y variables numéricas

para las variables categóricas aplicamos el OneHotEnconder transformandolas en un vector binario, se normaliza las variables para que no hayan tendencias significativas.

In [90]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

categorical_columns = ['SEX', 'EDUCATION', 'MARRIAGE']
numerical_columns = [col for col in x_train.columns if col not in categorical_columns]

preprocessor = make_column_transformer(
    (OneHotEncoder(), categorical_columns),
    (MinMaxScaler(), numerical_columns),
    remainder='passthrough'  # conservan cambios en columnas diferentes a las categóricas 
)


*Creación del pipeline*
- Se crea la "receta para entrenar el modelo en este caso una regresión logística
- Se aplica las transformaciones definidas en el preprocesador
- selecciona las 'k' carácteristicas más relevantes para predecir la variable objetivo 'default' ( k será definida más adelante)
- Reproducibilidad fijando una semilla aleatoria

In [91]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
 
pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ('feature_selection', SelectKBest(score_func = f_classif)),
    ('classifier', LogisticRegression(random_state = 42))
])

*Optimización del pipeline*

Especificamos los hiperparámetros y sus posibles valores:

- Características 1 a 10
- Regresión logística
- Penalización 
- Tipo de Algoritmo
- Número máximo de iteraciones

*Validación cruzada*
- Dividel los datos en 10 subcojuntos: 9 para entrenar y 1 para validad, rotando hasta usar todos como validación.
- Usa precisión balancead que es adecuada para datos desbalanceados
- Usar los núcleso de la CPU acelerando la búsqueda. para mi caso mi pc tiene 14 núcleos y 20 hilos (threads)
- Una vez encontrados los mejores hiperparámetros, se ajusta el pipeline completo con estos valores usando todo el conjunto de entrenamiento (refit=True)
- Entrenamos el modelo usando el conjunto de entreanamiento x_train y_train y realiza la búsqueda de hiperparámetros.


In [92]:
from sklearn.model_selection import GridSearchCV

model = GridSearchCV(
    estimator=pipeline,
    param_grid={
        'feature_selection__k': range(1, 11),
        'classifier__C': [0.1],
        'classifier__penalty': ['l1'],
        'classifier__solver': ['liblinear'],
        'classifier__max_iter': [100],
    },
    cv=10,
    scoring="balanced_accuracy",
    n_jobs=-1,
    refit=True
).fit(x_train, y_train)

print(f"Mejores hiperparámetros: {model.best_params_}")

Mejores hiperparámetros: {'classifier__C': 0.1, 'classifier__max_iter': 100, 'classifier__penalty': 'l1', 'classifier__solver': 'liblinear', 'feature_selection__k': 1}



*Guardamos el modelo hecho*

In [93]:
# Importar las librerías necesarias
import os
import gzip
import pickle

# Crear carpeta si no existe
models_dir = '../files/models'
os.makedirs(models_dir, exist_ok = True)

# Definir la ruta donde se guardará el modelo
model_path = "../files/models/model.pkl.gz"

# Guardar el modelo entrenado en un archivo
with gzip.open(model_path, "wb") as model_file:
    pickle.dump(model, model_file)

print(f"Modelo guardado en '{model_path}'")

Modelo guardado en '../files/models/model.pkl.gz'


* Métricas de evaluación y matrices de confusión*

Este código calcula métricas de evaluación y matrices de confusión para el modelo entrenado y guarda los resultados en un archivo JSON línea por línea.


- Definimos la función que calcula las métricas  "calculate_metrics" la cual comprar la etiquetas reales  y_true vs y_pred, especificando con dataset_type si las métricas calculadas pertenencen al conjunto de entrenamiento o prueba

Cada métrica se calcula utilizando las funciones de sklearn.metrics : Precisión, Precisión balanceada, recall,F1-score.

Definimos la función para calcular las matrices de confusión y así comprender el rendimiento del modelo en cada clase.
La idea es calcular los verdaderos negativos, Falsos positivos, Falsos negativos, verdaderos positivos
Utiliza el modelo entrenado para predecir las etiquetas (y_train_pred, y_test_pred) en los conjuntos de entrenamiento x_train y prueba x_test

Después de definidas las dos funciones auxiliares procedemos a hacer las respectivas predicciones y a calcular las metricas y las matrices de confusión para cada dataset para guardarlos como .json en la carpeta output

In [97]:
import json
import os
from sklearn.metrics import (
    precision_score,
    recall_score,
    f1_score,
    balanced_accuracy_score,
    confusion_matrix,
)

# Función para calcular métricas generales
def calculate_metrics(y_true, y_pred, dataset_type):
    return {
        "type": "metrics",
        "dataset": dataset_type,
        "precision": precision_score(y_true, y_pred, zero_division=0),
        "balanced_accuracy": balanced_accuracy_score(y_true, y_pred),
        "recall": recall_score(y_true, y_pred, zero_division=0),
        "f1_score": f1_score(y_true, y_pred, zero_division=0),
    }

# Función para formatear matrices de confusión
def format_confusion_matrix(cm, dataset_type):
    return {
        "type": "cm_matrix",
        "dataset": dataset_type,
        "true_0": {"predicted_0": int(cm[0, 0]), "predicted_1": int(cm[0, 1])},
        "true_1": {"predicted_0": int(cm[1, 0]), "predicted_1": int(cm[1, 1])},
    }

# Hacer predicciones
y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)

# Calcular métricas y matrices de confusión
entries = [
    calculate_metrics(y_train, y_train_pred, "train"),
    calculate_metrics(y_test, y_test_pred, "test"),
    format_confusion_matrix(confusion_matrix(y_train, y_train_pred), "train"),
    format_confusion_matrix(confusion_matrix(y_test, y_test_pred), "test"),
]

# Crear carpeta de salida y guardar métricas línea por línea
os.makedirs("../files/output", exist_ok=True)
with open("../files/output/metrics.json", "w") as f:
    for entry in entries:
        f.write(json.dumps(entry) + "\n")

print("Métricas y matrices de confusión guardadas exitosamente.")

Métricas y matrices de confusión guardadas exitosamente.
